In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt

Here is an example where I wanted to calculate the length of the river within each municipality. The following script performs these tasks:

- Reads the river as a line shapefile. Since my dataset is inconsistent and sometimes the river is divided into multiple lines, I used the dissolve function to merge them into a single line.
- Reads the municipality shapefile.
- Checks if both files have the same projection. If not, the script changes the line projection to match the polygon shapefile's projection.
- Plots the two layers to ensure they overlay correctly.
- Calculates the length of the river within each municipality and adds this information to the municipality shapefile.


In [ ]:
def add_river_lengths_to_polygons(lines_shapefile, polygons_shapefile, municipality_column):
    """
    This function reads the provided line and polygon shapefiles, dissolves the line shapefile into a single line,
    checks and potentially reprojects the line shapefile to match the CRS of the polygons, performs an overlay operation
    to determine where the river intersects with each municipality polygon, calculates the length of river within each
    polygon, and adds this information as a new column ('River_length_in_polygon') to the polygon shapefile. It uses
    the specified municipality column to merge the intersection results with the polygon layer. Additionally, it computes
    the cumulative river length ('Cumulative_River_length') within each polygon.

    Parameters:
    - lines_shapefile (str): File path to the line shapefile containing the river.
    - polygons_shapefile (str): File path to the polygon shapefile containing municipality boundaries.
    - municipality_column (str): Name of the column in polygons_shapefile that identifies municipalities.

    Returns:
    - GeoDataFrame: GeoDataFrame containing polygons with added columns for river length within each municipality
      and cumulative river length.
    """
    # Read the line shapefile
    lines = gpd.read_file(lines_shapefile)
    
    # Dissolve the merged GeoDataFrame into a single line
    lines = lines.dissolve()
    
    # Read the polygon shapefile
    polygons = gpd.read_file(polygons_shapefile)
    
    # Check if the CRS of the polygons and lines are the same
    if polygons.crs == lines.crs:
        print("The CRS of the polygons and lines are already the same.")
    else:
        # Reproject the lines to match the CRS of the polygons
        lines = lines.to_crs(polygons.crs)
        print("Lines have been reprojected to match the CRS of the polygons.")
    
    # Plotting for visual check
    ax = polygons.boundary.plot()
    lines.plot(ax=ax, color='red')
    
    # Perform overlay operation to get the intersection
    intersection = gpd.overlay(lines, polygons, how='intersection')
    
    # Calculate the length of the intersected lines within each polygon
    intersection['River_length_in_polygon'] = intersection.geometry.length
    
    # Merge the DataFrames based on the specified municipality column
    polygons = polygons.merge(intersection[[municipality_column, 'River_length_in_polygon']], on=municipality_column, how='left')
    
    # Calculate the cumulative sum
    polygons['Cumulative_River_length'] = polygons['River_length_in_polygon'].cumsum()
    
    return polygons
    
result = add_river_lengths_to_polygons(lines_shapefile, polygons_shapefile, municipality_column)

# Example usage:
lines_shapefile = r'path_to_your_lines_shapefile.shp'  # Update with your actual file path
polygons_shapefile = r'path_to_your_polygons_shapefile.shp'  # Update with your actual file path
municipality_column = 'gmde_sch'  # Update with a column from the municipality polygon layer it is going to be used to add the calcuated lengths to the polygon shapefile

In [ ]:
result